In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import joblib
from sklearn import metrics
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression



In [2]:
df = pd.read_pickle('../../data.scenarioTest2v2.pkl.gz')


In [ ]:
def pipeline(df):
    X = df.drop('winner', axis=1)
    y = df['winner']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    rf = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier())])
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print(y_pred)


In [30]:
def pipeline_save(df):
    X = df.drop('winner', axis=1)
    y = df['winner']    
    #categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), c)])
    rf = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier())])
    rf.fit(X, y)
    return(rf)


In [ ]:
def pipelineClassifier(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        pipe.fit(X_train, y_train)
        y_pred=pipe.predict(X_test)
        print(classifier)
        print("model score: %.3f" % pipe.score(X_test, y_test))
        print("model accuracy:",metrics.accuracy_score(y_test,y_pred))
        #print("model f1 score:",metrics.f1_score(y_test,y_pred))

In [3]:
def pipelineClassifierCross(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    cv = KFold(n_splits=10, random_state=1, shuffle=True)


    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print(classifier)
        print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
     







In [ ]:
pipelineClassifier(df)

In [ ]:
pipelineClassifierCross(df)

Accuracy: 0.638 (0.011)


In [29]:
pipeline_save(df)

ValueError: not enough values to unpack (expected 3, got 2)

In [31]:
joblib.dump(pipeline_save(df), 'model.joblib')


['model.joblib']

In [32]:
model=joblib.load('model.joblib')

In [18]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['team_red_2_0', 'name_red_2_0', 'stat-name_red_2_0', 'team_red_2_1',
       'name_red_2_1', 'stat-name_red_2_1', 'team_blue_2_0', 'name_blue_2_0',
       'stat-name_blue_2_0', 'team_blue_2_1', 'name_blue_2_1',
       'stat-name_blue_2_1', 'meta_scenario', 'meta_p_red', 'meta_p_blue'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [26]:
df.sample(1).drop('pippo',1,errors="ignore")

,team_red_2_0,name_red_2_0,index_red_2_0,kind_red_2_0,move_red_2_0,load_red_2_0,hp_red_2_0,hp_max_red_2_0,int_atk_red_2_0,int-def_red_2_0,...,weapon_AT_blue_2_1,weapon_MT_blue_2_1,weapon_GR_blue_2_1,weapon_SM_blue_2_1,weapon_SR_blue_2_1,winner,meta_scenario,meta_p_red,meta_p_blue,meta_seed
2,red,Tank1,0,2,7,1,1,1,6,0,...,0,0,0,2,0,red,scenarioTest2v2,GreedyAgent,RandomAgent,737181267


In [36]:
model.predict_proba(df.sample(1).drop('winner',1))

array([[0.69071656, 0.30928344]])